# Collaborative-Based Filtering

In collaborative-based filtering, we aim to predict what content a user might enjoy by identifying similar users and their preferences for movies. Since not all users rate every movie, we predict how a user would rate unseen movies based on their past preferences and those of similar users. This prediction is made through training a machine learning model on a dataset of user IDs and movie ratings.

Once trained, the model can predict a user's rating for a particular movie based on their user ID and the movie. If the predicted rating is high, we recommend the movie to that user. To implement this, we organize the dataset into a training set and train the model. This allows us to efficiently cater to diverse user preferences by providing personalized recommendations.

## Load data

In [27]:
import pandas as pd

# Load the ratings data from the 'ratings.csv' file into a DataFrame
ratings_df = pd.read_csv('./data/ratings.csv')

# Selecting only the columns 'userId', 'movieId', and 'rating' from the DataFrame
ratings_df = ratings_df.loc[:, ['userId', 'movieId', 'rating']]

# Display the first few rows of the ratings DataFrame to get an overview of the data
ratings_df.head()

,userId,movieId,rating
0,1,31,2.5
1,1,1029,3.0
2,1,1061,3.0
3,1,1129,2.0
4,1,1172,4.0


## Create the dataset

In [3]:
import surprise

In [28]:
from surprise import Dataset, Reader

# Creating a Reader object with the specified rating scale
# The rating scale indicates the minimum and maximum possible ratings in the dataset
# Here, the ratings range from 1 to 5
reader = Reader(rating_scale=(1, 5))

# Loading the dataset from a DataFrame using Surprise's load_from_df() function
# The ratings_df DataFrame contains the ratings data with columns 'userId', 'movieId', and 'rating'
# The Reader object is passed to specify the rating scale
dataset = Dataset.load_from_df(ratings_df, reader)

## Build the trainset

In [29]:
# Building the training set from the dataset
# The build_full_trainset() function constructs a training set that contains all the ratings from the original dataset
# This is useful for training models on the entire available data
# It creates a Trainset object, which is a special data structure used by Surprise for training models
trainset = dataset.build_full_trainset()

In [30]:
# Converting all ratings in the training set to a list and selecting the first 10 ratings
# The all_ratings() method returns a generator that yields tuples of (user_id, item_id, rating) for all ratings in the training set
# By converting the generator to a list, we can easily manipulate and view the ratings
# We use list slicing [:10] to select the first 10 ratings from the list
list(trainset.all_ratings())[:10]

[(0, 0, 2.5),
 (0, 1, 3.0),
 (0, 2, 3.0),
 (0, 3, 2.0),
 (0, 4, 4.0),
 (0, 5, 2.0),
 (0, 6, 2.0),
 (0, 7, 2.0),
 (0, 8, 3.5),
 (0, 9, 2.0)]

## Train the Model

In [31]:
# Importing the SVD (Singular Value Decomposition) model from Surprise
from surprise import SVD

# Creating an instance of the SVD model
svd = SVD()

# Fitting the SVD model to the training set
# The fit() method trains the model using the data in the training set
svd.fit(trainset)

In [32]:
# Make a prediction for user 15 and item 1956
# This predicts the rating that user 15 would give to item 1956
prediction = svd.predict(15, 1956)

prediction

Prediction(uid=15, iid=1956, r_ui=None, est=3.01651083732151, details={'was_impossible': False})

In [33]:
# Retrieve the estimated rating from the prediction object
# The 'est' attribute of the prediction object contains the estimated rating
estimation = prediction.est

estimation

3.01651083732151

## Validation

In [34]:
from surprise import model_selection

# Perform cross-validation to evaluate the performance of the SVD model using RMSE and MAE measures
# 'svd' is the trained Singular Value Decomposition (SVD) model
# 'dataset' is the dataset used for training and testing the model
# 'measures' specifies the evaluation metrics to be computed, here we use Root Mean Squared Error (RMSE) and Mean Absolute Error (MAE)
# Cross-validation divides the dataset into multiple folds, trains the model on some folds, 
# and tests it on others to get an overall estimate of performance
accuracy = model_selection.cross_validate(svd, dataset, measures=['RMSE', 'MAE'])

accuracy

{'test_rmse': array([0.9011104 , 0.88778282, 0.89612143, 0.8968526 , 0.90322592]),
 'test_mae': array([0.69659455, 0.68170277, 0.69003976, 0.68924303, 0.695387  ]),
 'fit_time': (1.1889941692352295,
  1.2052981853485107,
  1.2481846809387207,
  1.1065196990966797,
  1.2124087810516357),
 'test_time': (0.14124774932861328,
  0.2852060794830322,
  0.13715481758117676,
  0.12125349044799805,
  0.2573988437652588)}